# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [31]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [32]:
# Get current folder and subfolder event data

filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [33]:
# create a list of paths for looping through when appending the files

full_data_rows_list = []
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 

# Create smaller event data csv file used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [34]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

411


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [35]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [36]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity 
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [37]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [38]:
# Create song_listening history with session_id as the partition key item_in_session as the clustering column. The data will be \
# distributed across nodes using the session_id which is the partition key

session.execute("drop table if exists song_listening_history;")

query = "CREATE TABLE IF NOT EXISTS song_listening_history "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"

session.execute(query)

In [39]:
# insert data to created table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listening_history (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [40]:
query = "SELECT artist_name, song_title, song_length FROM song_listening_history WHERE session_id=338 AND item_in_session=4"

rows = session.execute(query)
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182                    

In [41]:
# Create table (artist_listening_history) with user_id and session_id as composite partitions keys. 
# Columns are ordered by partition key, clustering key then other required fields
session.execute("drop table if exists artist_listening_history")

query = "CREATE TABLE IF NOT EXISTS artist_listening_history "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id),  item_in_session))"

session.execute(query)

In [42]:
# Insert data to table created above

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_listening_history (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [43]:
# Verify that the data was entered into the table

query = "SELECT artist_name, song_title, first_name, last_name FROM artist_listening_history WHERE user_id=10 AND session_id=182"

rows = session.execute(query)
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'                  

In [44]:
# Create a table (user_listening_history) with song_tile and user_id as the primary key. 
# The column order is based on partition key, clustering key then other requried values

session.execute("drop table if exists user_listening_history")

query = "CREATE TABLE IF NOT EXISTS user_listening_history "
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"

session.execute(query)

In [45]:
# insert data to created table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listening_history (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"       
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [46]:
# Confirm that data was successfully inserted to the table

query = "SELECT first_name, last_name FROM user_listening_history WHERE song_title='All Hands Against His Own'"

rows = session.execute(query)
for row in rows:
    print(row.first_name, row.last_name)

Sara Johnson


### Drop the tables before closing out the sessions

In [47]:
session.execute("drop table if exists song_listening_history")
session.execute("drop table if exists artist_listening_history")
session.execute("drop table if exists user_listening_history")

### Close the session and cluster connection¶

In [48]:
session.shutdown()
cluster.shutdown()